<img width="8%" alt="Spotify.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/Spotify.png" style="border-radius: 15%">

# Spotify - Get User Top Items
<a href="https://bit.ly/3JyWIk6">Give Feedback</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title=Spotify+-+Get+User+Top+Items:+Error+short+description">Bug report</a>

**Tags:** #spotify #api #gettopitems #webapi #reference #snippet

**Author:** [Alton Liew](https://www.linkedin.com/in/alton-liew-749944182/)

**Last update:** 2023-12-14 (Created: 2023-12-10)

**Description:** This notebook retrieve the current user's top artists or tracks based on calculated affinity.

**References:**
- [Spotify Web API Reference - Get User Top Items](https://developer.spotify.com/documentation/web-api/reference/get-users-top-artists-and-tracks)
- [Spotify Developer Documentation](https://developer.spotify.com/documentation/)

## Input

### Import libraries

In [ ]:
try:
    import spotipy
except:
    !pip install spotipy --user --upgrade
    import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import naas
from IPython.display import Image, display
from spotipy.oauth2 import SpotifyOAuth

### Setup variables
- `client_id`: Your Spotify API client ID. [Get your client ID](https://developer.spotify.com/documentation/general/guides/app-settings/#register-your-app)
- `client_secret`: Your Spotify API client secret. [Get your client secret](https://developer.spotify.com/documentation/general/guides/app-settings/#register-your-app)
- `redirect_uri`: Your Spotify API redirect uri. This redirects users to a uri for authentication. [Get your redirect uri](https://developer.spotify.com/documentation/general/guides/app-settings/#register-your-app)
- `limit`: The amount of results you want to see.
- `time_range`: Over what time frame are the affinities computed. Valid-values: `short_term`, `medium_term`, `long_term`

In [ ]:
client_id = "YOUR_SPOTIFY_CLIENT_ID"
client_secret = "YOUR_SPOTIFY_CLIENT_SECRET"
redirect_uri = "YOUR_SPOTIFY_REDIRECT_URI"

limit = 5
time_range = "medium_term"

## Model

### Get user top items
* Retrieve current logged in user's top items both tracks and artists.
* Authenticates the user by redirecting the user to the uri and user must then authorize and copy the code in the URL and paste in the input.
* Once user is authenticated, `fetch_top_items` can not print the top items.

In [ ]:
def authenticate_spotify(client_id, client_secret, redirect_uri):
    # Create a Spotify API client
    sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=client_id, client_secret=client_secret, redirect_uri=redirect_uri, scope="user-library-read user-top-read"))

    # Get the authorization URL
    auth_url = sp.auth_manager.get_authorize_url()

    # Prompt the user to visit the authorization URL
    print(f"Please visit this URL to authorize the application: {auth_url}")

    # Ask the user to input the authorization code
    authorization_code = input("Enter the authorization code from the URL: ")

    # Exchange authorization code for access token
    token_info = sp.auth_manager.get_access_token(authorization_code)

    # Check authentication status
    if sp.auth_manager.get_cached_token() is not None:
        print("Successfully authenticated")
        return sp  # Return the Spotify client for further use
    else:
        print("Authentication failed")
        return None
    
def fetch_top_items(sp, limit, time_range):
    try:
        # Top tracks
        print("\nTop tracks:")
        top_tracks = sp.current_user_top_tracks(limit=limit, time_range=time_range)
        for idx, track in enumerate(top_tracks['items'], 1):
            print(f"{idx}. {track['name']} by {track['artists'][0]['name']}, Track ID: {track['id']}")
            display(Image(url=track['album']['images'][0]['url'], width=200))
            print("\n")
            
        # Top artists
        print("\nTop artists:")
        top_artists = sp.current_user_top_artists(limit=limit, time_range=time_range)
        for idx, artist in enumerate(top_artists['items'], 1):
            print(f"{idx}. {artist['name']}, Artist ID: {artist['id']}")
            display(Image(url=artist['images'][0]['url'], width=200))
            print("\n")

    except spotipy.SpotifyException as e:
        print(f"Spotify API error: {e}")
    except Exception as e:
        print(f"Error: {e}")

## Output

### Display result
* Calls the `authenticate_spotify` function to authenticate user then stores the client in the `spotify_client` variable.
* If `spotify_client` is available, then pass the client into `fetch_top_items` function and print out relevant information.

In [ ]:
spotify_client = authenticate_spotify(client_id, client_secret, redirect_uri)
if spotify_client:
    fetch_top_items(spotify_client, limit, time_range)